In [ ]:
# Comandos mágicos para recargar cambios en tu script sin reiniciar el kernel
%load_ext autoreload
%autoreload 2

import sys
import os
import polars as pl


# ".." significa subir un nivel (salir de notebooks) y entrar a scripts
ruta_scripts = os.path.abspath(os.path.join('..', 'src'))
if ruta_scripts not in sys.path:
    sys.path.append(ruta_scripts)

# Ahora importamos tu función
try:
    import etl_tools as etl
    print("Librería etl_tools importada correctamente.")
except ImportError as e:
    print(f"Error: No se encuentra el archivo etl_tools.py en {ruta_scripts}")

In [ ]:
#---IMPORTACIONES ----- 

#Rutas de entrada
ruta_zips_impo = os.path.join('..', 'Data', 'Impo')
ruta_zips_expo = os.path.join('..', 'Data', 'Expo')

# Rutas de salida 
lake_impo = os.path.join('..','Data', 'datalake_impo')
lake_expo = os.path.join('..', 'Data', 'datalake_expo')

#----Columnas que usaremos de importaciones------
cols_impo = ['FECH', 'PAISGEN' , 'VAFODO', 'PBK', 'ADUA', 'NABAN', 'REGIMEN']
txt_impo  = ['FECH', 'PAISGEN', 'ADUA', 'NABAN', 'REGIMEN']
num_impo  = ['VAFODO', 'PBK']

#----Columnas que usaremos de exportaciones------
cols_expo = ['FECH', 'COD_PAI4' , 'FOBDOL', 'PBK', 'ADUA', 'POSAR', 'MODAD'] 
txt_expo  = ['FECH', 'COD_PAI4' , 'ADUA', 'POSAR' , 'MODAD']
num_expo  = ['FOBDOL', 'PBK']


print("Configuración lista.")

In [ ]:
print("🚀 INICIANDO CONSTRUCCIÓN DE DATA LAKES")
print("="*40)

# 1. Procesar IMPORTACIONES
print(f"\n📦 Procesando Importaciones...")
etl.zips_a_parquet(
    ruta_origen=ruta_zips_impo,
    ruta_destino=lake_impo,
    columnas_necesarias=cols_impo,
    cols_texto=txt_impo,
    cols_numero=num_impo
)

print("-" * 40)

# 2. Procesar EXPORTACIONES
print(f"\n📦 Procesando Exportaciones...")

try:
    etl.zips_a_parquet(
        ruta_origen=ruta_zips_expo,
        ruta_destino=lake_expo,
        columnas_necesarias=cols_expo,
        cols_texto=txt_expo,
        cols_numero=num_expo
    )
except Exception as e:
    print(f"⚠️ Error en Exportaciones: {e}")
    print("Consejo: Abre un archivo .dta de expo individualmente para verificar los nombres exactos de las columnas.")

print("\n✨ ¡PROCESO FINALIZADO!")

In [ ]:
# Verificación rápida
print("--- Auditoría de Datos ---")

try:
    # Chequeo Impo
    df_impo = pl.scan_parquet(os.path.join(lake_impo, "*.parquet"))
    conteo_impo = df_impo.select(pl.len()).collect().item()
    print(f"✅ Data Lake Importaciones: {conteo_impo:,.0f} registros accesibles.")
    
    # Chequeo Expo
    df_expo = pl.scan_parquet(os.path.join(lake_expo, "*.parquet"))
    conteo_expo = df_expo.select(pl.len()).collect().item()
    print(f"✅ Data Lake Exportaciones: {conteo_expo:,.0f} registros accesibles.")

except Exception as e:
    print(f"❌ Algo salió mal al leer los parquets: {e}")

In [ ]:
mapa_dian_impo_nombres = {
    "013": "AFGANISTAN", "017": "ALBANIA", "023": "ALEMANIA", "026": "ARMENIA",
    "027": "ARUBA", "029": "BOSNIA-HERZEGOVINA", "031": "BURKINA FASSO", "037": "ANDORRA",
    "040": "ANGOLA", "041": "ANGUILLA", "043": "ANTIGUA Y BARBUDA", "047": "ANTILLAS HOLANDESAS",
    "053": "ARABIA SAUDITA", "059": "ARGELIA", "063": "ARGENTINA", "069": "AUSTRALIA",
    "072": "AUSTRIA", "074": "AZERBAIJAN", "077": "BAHAMAS", "080": "BAHREIN",
    "081": "BANGLADESH", "083": "BARBADOS", "087": "BELGICA", "088": "BELICE",
    "090": "BERMUDAS", "091": "BELARUS", "093": "BIRMANIA (MYANMAR)", "097": "BOLIVIA",
    "101": "BOTSWANA", "105": "BRASIL", "108": "BRUNEI DARUSSALAM", "111": "BULGARIA",
    "115": "BURUNDI", "119": "BUTAN", "127": "CABO VERDE", "137": "CAIMAN, ISLAS",
    "141": "CAMBOYA (KAMPUCHEA)", "145": "CAMERUN, REPUBLICA UNIDA DEL", "149": "CANADA", "159": "SANTA SEDE",
    "165": "COCOS (KEELING), ISLAS", "169": "COLOMBIA", "173": "COMORAS", "177": "CONGO",
    "183": "COOK, ISLAS", "187": "COREA (NORTE), REPUBLICA POPULAR DEMOCRATICA DE", 
    "190": "COREA (SUR), REPUBLICA DE", "193": "COSTA DE MARFIL", "196": "COSTA RICA", "198": "CROACIA",
    "199": "CUBA", "203": "CHAD", "211": "CHILE", "215": "CHINA", "218": "TAIWAN (FORMOSA)",
    "221": "CHIPRE", "229": "BENIN", "232": "DINAMARCA", "235": "DOMINICA", "239": "ECUADOR",
    "240": "EGIPTO", "242": "EL SALVADOR", "243": "ERITREA", "244": "EMIRATOS ARABES UNIDOS",
    "245": "ESPAÑA", "246": "ESLOVAQUIA", "247": "ESLOVENIA", "249": "ESTADOS UNIDOS",
    "251": "ESTONIA", "253": "ETIOPIA", "259": "FEROE, ISLAS", "267": "FILIPINAS",
    "271": "FINLANDIA", "275": "FRANCIA", "281": "GABON", "285": "GAMBIA", "287": "GEORGIA",
    "289": "GHANA", "293": "GIBRALTAR", "297": "GRANADA", "301": "GRECIA", "305": "GROENLANDIA",
    "309": "GUADALUPE", "313": "GUAM", "317": "GUATEMALA", "325": "GUAYANA FRANCESA",
    "329": "GUINEA", "331": "GUINEA ECUATORIAL", "334": "GUINEA-BISSAU", "337": "GUYANA",
    "341": "HAITI", "345": "HONDURAS", "351": "HONG KONG", "355": "HUNGRIA", "361": "INDIA",
    "365": "INDONESIA", "369": "IRAK", "372": "IRAN, REPUBLICA ISLAMICA DEL", "375": "IRLANDA (EIRE)",
    "379": "ISLANDIA", "383": "ISRAEL", "386": "ITALIA", "391": "JAMAICA", "399": "JAPON",
    "403": "JORDANIA", "406": "KAZAJSTAN", "410": "KENIA", "411": "KIRIBATI",
    "412": "KIRGUIZISTAN", "413": "KUWAIT", "420": "LAOS, REPUBLICA POPULAR DEMOCRATICA DE", 
    "426": "LESOTHO", "429": "LETONIA", "431": "LIBANO", "434": "LIBERIA",
    "438": "LIBIA (INCLUYE FEZZAN)", "440": "LIECHTENSTEIN", "443": "LITUANIA", "445": "LUXEMBURGO",
    "447": "MACAO", "448": "MACEDONIA", "450": "MADAGASCAR", "455": "MALAYSIA",
    "458": "MALAWI", "461": "MALDIVAS", "464": "MALI", "467": "MALTA",
    "469": "MARIANAS DEL NORTE, ISLAS", "472": "MARSHALL, ISLAS", "474": "MARRUECOS", "477": "MARTINICA",
    "485": "MAURICIO", "488": "MAURITANIA", "493": "MEXICO", "494": "MICRONESIA, ESTADOS FEDERADOS DE",
    "496": "MOLDAVIA", "497": "MONGOLIA", "498": "MONACO", "501": "MONSERRAT, ISLA",
    "505": "MOZAMBIQUE", "507": "NAMIBIA", "508": "NAURU", "511": "NAVIDAD (CHRISTMAS), ISLAS",
    "517": "NEPAL", "521": "NICARAGUA", "525": "NIGER", "528": "NIGERIA", "531": "NIUE, ISLA",
    "535": "NORFOLK, ISLA", "538": "NORUEGA", "542": "NUEVA CALEDONIA", "545": "PAPUASIA NUEVA GUINEA",
    "548": "NUEVA ZELANDIA", "551": "VANUATU", "556": "OMAN", "566": "PACIFICO, ISLAS (USA)",
    "573": "PAISES BAJOS (HOLANDA)", "576": "PAKISTAN", "578": "PALAU, ISLAS", "580": "PANAMA",
    "586": "PARAGUAY", "589": "PERU", "593": "PITCAIRN, ISLA", "599": "POLINESIA FRANCESA",
    "603": "POLONIA", "607": "PORTUGAL", "611": "PUERTO RICO", "618": "QATAR",
    "628": "REINO UNIDO", "640": "REPUBLICA CENTROAFRICANA", "644": "REPUBLICA CHECA", 
    "647": "REPUBLICA DOMINICANA", "660": "REUNION", "665": "ZIMBABWE", "670": "RUMANIA",
    "675": "RUANDA", "676": "RUSIA", "677": "SALOMON, ISLAS", "685": "SAHARA OCCIDENTAL",
    "687": "SAMOA", "690": "SAMOA NORTEAMERICANA", "695": "SAN CRISTOBAL Y NIEVES", 
    "697": "SAN MARINO", "700": "SAN PEDRO Y MIGUELON", "705": "SAN VICENTE Y LAS GRANADINAS", 
    "710": "SANTA ELENA", "715": "SANTA LUCIA", "720": "SANTO TOME Y PRINCIPE", "728": "SENEGAL",
    "729": "SERBIA", "731": "SEYCHELLES", "735": "SIERRA LEONA", "741": "SINGAPUR",
    "744": "SIRIA, REPUBLICA ARABE DE", "748": "SOMALIA", "750": "SRI LANKA", 
    "756": "SUDAFRICA, REPUBLICA DE", "759": "SUDAN", "764": "SUECIA", "767": "SUIZA",
    "770": "SURINAM", "773": "SWAZILANDIA", "774": "TADJIKISTAN", "776": "TAILANDIA",
    "780": "TANZANIA, REPUBLICA UNIDA DE", "783": "DJIBOUTI", 
    "787": "TERRITORIO BRITANICO DEL OCEANO INDICO", "788": "TIMOR DEL ESTE", "800": "TOGO",
    "805": "TOKELAU", "810": "TONGA", "815": "TRINIDAD Y TOBAGO", "820": "TUNICIA",
    "823": "TURCAS Y CAICOS, ISLAS", "825": "TURKMENISTAN", "827": "TURQUIA", "828": "TUVALU",
    "830": "UCRANIA", "833": "UGANDA", "845": "URUGUAY", "847": "UZBEKISTAN",
    "850": "VENEZUELA", "855": "VIETNAM", "863": "VIRGENES, ISLAS (BRITANICAS)", 
    "866": "VIRGENES, ISLAS (NORTEAMERICANAS)", "870": "FIJI", "875": "WALLIS Y FORTUNA, ISLAS",
    "880": "YEMEN", "885": "YUGOSLAVIA", "888": "ZAIRE", "890": "ZAMBIA",
    "897": "ZONA NEUTRAL PALESTINA", "911": "ZONA FRANCA DE BARRANQUILLA", 
    "912": "ZONA FRANCA DE BUENAVENTURA", "913": "ZONA FRANCA DE PALMASECA - CALI", 
    "914": "ZONA FRANCA DE CUCUTA", "915": "ZONA FRANCA DE SANTA MARTA", 
    "916": "ZONA FRANCA DE CARTAGENA", "917": "ZONA FRANCA DE RIONEGRO MEDELLIN", 
    "918": "ZONA FRANCA DE CANDELARIA - CARTAGENA", "919": "ZONA FRANCA DE BOGOTA", 
    "920": "ZONA FRANCA DE PACIFICO - CALI", "921": "ZONA FRANCA DE BARU BEACH RESORT", 
    "922": "ZONA FRANCA DE POZOS COLORADOS", "923": "ZONA FRANCA DE EUROCARIBE DE INDIAS", 
    "924": "ZONA FRANCA DEL EJE CAFETERO", "928": "ZFP La Cayena", "999": "NO DECLARADOS"
}

In [ ]:
import plotly.express as px
import polars as pl 

# 1. CONEXIÓN AL DATALAKE (Modo Lazy - No consume RAM)
# ----------------------------------------------------
ruta_parquets = r"C:\Users\thoma\Documentos\Analisis de Datos\Portafolio Proyectos\Expo e Impo Colombia\Data\datalake_impo"
lazy_df = pl.scan_parquet(ruta_parquets)

print("Datos conectados. Generando gráficos...")

In [ ]:
# ==============================================================================
# GRÁFICO 1: EVOLUCIÓN TEMPORAL (Línea)
# ==============================================================================
print("1. Procesando Serie de Tiempo...")

# Asumimos que FECH viene formato 'YYYYMM' o similar.
# Extraemos el Año para agrupar.
df_tiempo = (
    lazy_df
    .with_columns(
        pl.col("ANIO").alias("ANIO") 
    )
    .group_by("ANIO")
    .agg(pl.col("VAFODO").sum().alias("VALOR_TOTAL"))
    .sort("ANIO")
    .collect() # <--- Aquí Polars procesa los 23M de filas
)

# Graficamos con Plotly
fig_line = px.line(
    df_tiempo.to_pandas(), 
    x="ANIO", 
    y="VALOR_TOTAL", 
    title="Evolución Histórica de Importaciones (Valor FOB)",
    markers=True,
    labels={"VALOR_TOTAL": "Dólares (USD)", "ANIO": "Año"}
)
fig_line.show()


In [ ]:
# ==============================================================================
# GRÁFICO 2: TOP 10 PAÍSES DE ORIGEN (Barras)
# ==============================================================================
print("2. Procesando Top Países...")

df_paises = (
    lazy_df
    .group_by("PAISGEN")
    .agg(pl.col("VAFODO").sum().alias("VALOR_TOTAL"))
    .sort("VALOR_TOTAL", descending=True)
    .limit(10) 
    .collect() # Ejecutamos para traer los 10 datos a memoria
)

# --- APLICAR MAPEO ISO ---
df_paises = df_paises.with_columns(
    pl.col("PAISGEN")
    .cast(pl.String)
    .str.replace(".0", "", literal=True) # Limpieza segura
    .replace(mapa_dian_impo_nombres)              # Conversión a ISO (Ej: 249 -> USA)
    .alias("ISO_CODE")                   # Guardamos en columna nueva
)

# Convertimos a Pandas para Plotly
pdf_paises = df_paises.to_pandas()

# Graficamos usando 'ISO_CODE' en el eje X
fig_bar = px.bar(
    pdf_paises,
    x="ISO_CODE",      # <--- Ahora usamos el código ISO
    y="VALOR_TOTAL",
    title="Top 10 Países de Origen (ISO)",
    text_auto='.2s',   # Formato resumido (10B, 5M)
    color="VALOR_TOTAL",
    hover_data=["PAISGEN"], # Muestra el código original al pasar el mouse (opcional)
    labels={"ISO_CODE": "País (ISO)", "VALOR_TOTAL": "Valor FOB (USD)"}
)

fig_bar.update_layout(xaxis_title="País", yaxis_title="USD Total")
fig_bar.show()




In [ ]:
# Diccionario de conversión: Código DIAN (Texto) -> Código ISO Alpha-3
# Fuente: Códigos de Países DIAN vs ISO 3166-1 alpha-3
mapa_dian_iso = {
    "249": "USA", # Estados Unidos
    "215": "CHN", # China
    "493": "MEX", # México
    "072": "BRA", # Brasil
    "105": "DEU", # Alemania
    "169": "COL", # Colombia (Reimportaciones)
    "361": "IND", # India
    "239": "ECU", # Ecuador
    "275": "FRA", # Francia
    "399": "JPN", # Japón
    "063": "ARG", # Argentina
    "410": "KOR", # Corea del Sur
    "385": "ITA", # Italia
    "245": "ESP", # España
    "589": "PER", # Perú
    "149": "CAN", # Canadá
    "521": "NLD", # Países Bajos (Holanda)
    "827": "TUR", # Turquía
    "850": "VEN", # Venezuela
    "196": "CRI", # Costa Rica
    "152": "CHL", # Chile
    "767": "CHE", # Suiza
    "809": "SWE", # Suecia
    "628": "GBR", # Reino Unido
    "069": "AUS", # Australia
    "607": "PRT", # Portugal
    "059": "BEL", # Bélgica
    "880": "VNM", # Vietnam
    "379": "IDN", # Indonesia
    "744": "SGP", # Singapur
    "764": "THA", # Tailandia
    "450": "MYS", # Malasia
    "603": "POL", # Polonia
    "687": "RUS", # Rusia
    "813": "TWN", # Taiwán
    "375": "HKG", # Hong Kong
    "240": "EGY", # Egipto
    "576": "PAK", # Pakistán
    "244": "ARE", # Emiratos Árabes Unidos
    "391": "ISR", # Israel
    "053": "SAU"  # Arabia Saudita
}
# ==============================================================================
# GRÁFICO 3: MAPA MUNDIAL (Choropleth)
# ==============================================================================
print("3. Procesando Datos para el Mapa...")

# Agrupamos por país (todos, no solo el top 10)
df_mapa = (
    lazy_df
    .group_by("PAISGEN")
    .agg(pl.col("VAFODO").sum().alias("VALOR_TOTAL"))
    .collect()
)

# 2. Aplicar el mapeo

df_mapa = df_mapa.with_columns(
    pl.col("PAISGEN")
    .cast(pl.String)             # Asegurar que sea texto
    .replace(mapa_dian_iso)      # <--- AQUÍ OCURRE LA MAGIA
    .alias("ISO_CODE")           # Nueva columna para Plotly
)

# 3. Filtrar los que no encontraron mapeo (quedan igual al código original)
# Para que el mapa no falle, nos quedamos solo con los que tienen formato ISO (3 letras)
# Opcional: imprimir los que faltaron para agregarlos al diccionario luego
faltantes = df_mapa.filter(pl.col("ISO_CODE").str.len_chars() != 3)
if not faltantes.is_empty():
    print(f"Advertencia: {len(faltantes)} códigos de país no se encontraron en el diccionario.")
    print(faltantes.head(5))

# 4. Convertir a Pandas para Plotly
pdf_mapa = df_mapa.to_pandas()

# 5. Graficar
fig_map = px.choropleth(
    pdf_mapa,
    locations="ISO_CODE",     # Usamos la columna mapeada
    color="VALOR_TOTAL",
    hover_name="PAISGEN",     # Muestra el código DIAN al pasar el mouse
    color_continuous_scale=px.colors.sequential.Plasma,
    title="Mapa de Importaciones (Origen)",
    labels={"VALOR_TOTAL": "Valor FOB (USD)"}
)

# Mejorar proyección (que no se vea plano)
fig_map.update_geos(projection_type="natural earth")
fig_map.show()

In [ ]:
rutas_parquets_expo = r"C:\Users\thoma\Documentos\Analisis de Datos\Portafolio Proyectos\Expo e Impo Colombia\Data\datalake_expo"
lazy_df_expo = pl.scan_parquet(rutas_parquets_expo)
print("Datos de Exportaciones conectados.")
print("Columnas disponibles:", lazy_df_expo.collect_schema())

In [ ]:
df_mapa = (
    lazy_df_expo
    .group_by("COD_PAI4")
    .agg(pl.col("FOBDOL").sum().alias("VALOR_TOTAL"))
    .collect()
)

# 2. Aplicar el mapeo

df_mapa = df_mapa.with_columns(
    pl.col("COD_PAI4")
    .cast(pl.String)             # Asegurar que sea texto
    .replace(mapa_dian_iso)      # <--- AQUÍ OCURRE LA MAGIA
    .alias("ISO_CODE")           # Nueva columna para Plotly
)

# 3. Filtrar los que no encontraron mapeo (quedan igual al código original)
# Para que el mapa no falle, nos quedamos solo con los que tienen formato ISO (3 letras)
# Opcional: imprimir los que faltaron para agregarlos al diccionario luego
faltantes = df_mapa.filter(pl.col("ISO_CODE").str.len_chars() != 3)
if not faltantes.is_empty():
    print(f"Advertencia: {len(faltantes)} códigos de país no se encontraron en el diccionario.")
    print(faltantes.head(5))

# 4. Convertir a Pandas para Plotly
pdf_mapa = df_mapa.to_pandas()

# 5. Graficar
fig_map = px.choropleth(
    pdf_mapa,
    locations="ISO_CODE",     # Usamos la columna mapeada
    color="VALOR_TOTAL",
    hover_name="COD_PAI4",     # Muestra el código DIAN al pasar el mouse
    color_continuous_scale=px.colors.sequential.Plasma,
    title="Mapa de Exportaciones (Origen)",
    labels={"VALOR_TOTAL": "Valor FOB (USD)"}
)

# Mejorar proyección (que no se vea plano)
fig_map.update_geos(projection_type="natural earth")
fig_map.show()